## Great, now that we discussed a little let's continue

Given that the current approach utilized by the authors lacks reproducibility, we will explore an alternative method by leveraging nf-core pipelines for data analysis.

Please explain, how we will achieve reproducibility for the course  with this approach.


Using nf-core pipelines ensures reproducibility for analyses. It does so by utilizing Nextflow, which allows the usage of Docker. With Docker containers the pipeline can be executed in the same computing environment.

Ewels, Philip A., et al. "The nf-core framework for community-curated bioinformatics pipelines." Nature biotechnology 38.3 (2020): 276-278.

You have successfully downloaded 2 of the fastq files we will use in our study.

What is the next step if we want to first have a count table and check the quality of our fastq files? What is the pipeline called to do so?

The next step is to use the nf-core pipeline rnaseq. It takes as input the output of the fetchngs pipeline and among other things computes a count table and does quality control.

https://nf-co.re/rnaseq/3.15.1/ (1.10.2024)

Analyze the 2 files using an nf-core pipeline.

What does this pipeline do?
Which are the main tools that will be used in the pipeline?

The pipeline analyses RNA sequencing data. The main tools used in the pipeline are:
- `Trim Galore!`for adapter and quality trimming
- `BBSplit`and `SortMeRNA`for the removal of genome contaminants and ribosomal RNA
- `STAR`, `HiSAT2`, `Salmon` and `RSEM` for producing alignments and quantification
- `RSeQC`, `Qualimap`, `dupRadar`, `Preseq`, and `DESeq2`for quality control

Other tools used by the pipeline are: `cat`,`fq`,`FastQC`,`UMI-tools`,`SAMtools`,`UMI-tools`,`picard MarkDuplicates`,`StringTie`,`BEDTools`,`bedGraphToBigWig`,`Kallisto`,`R`

https://nf-co.re/rnaseq/3.15.1/ (1.10.2024)

As all other nf-core pipelines, the chosen pipeline takes in a samplesheet as input.

Use Python and pandas to create the samplesheet for your 2 samples. Feel free to make use of the table you created earlier today.

Choose your sample names wisely, they must be the connection of the results to the metadata. If you can't find the sample in the metadata later, the analysis was useless.

In [4]:
# post here the command you used to run nf-core/rnaseq
# 
import pandas as pd

# modify samplesheet
samplesheet = pd.read_csv('DAY2_1/samplesheet/samplesheet.csv',delimiter = ',')   
samplesheet = samplesheet[['sample','fastq_1','fastq_2']]
samplesheet['strandedness'] = list(['auto','auto'])
samplesheet.to_csv('samplesheet_rnaseq.csv', sep=',', index=False)

!nextflow run nf-core/rnaseq -r 3.15.0  -profile docker --input samplesheet_rnaseq.csv --outdir DAY2_2 --genome GRCm38 --max_memory '7GB' --max_cpus 4 -resume

[sudo] Passwort für jana: 
 N E X T F L O W   ~  version 24.04.4

Launching `https://github.com/nf-core/rnaseq` [prickly_lamport] DSL2 - revision: 4e34945f6c [3.15.0]

WARN: Access to undefined parameter `monochromeLogs` -- Initialise it to a default value eg. `params.monochromeLogs = some_value`


------------------------------------------------------
                                        ,--./,-.
        ___     __   __   __   ___     /,-._.--~'
  |\ | |__  __ /  ` /  \ |__) |__         }  {
  | \| |       \__, \__/ |  \ |___     \`-._,-`-,
                                        `._,._,'
  nf-core/rnaseq v3.15.0-g4e34945
------------------------------------------------------
Core Nextflow options
  revision        : 3.15.0
  runName         : prickly_lamport
  containerEngine : docker
  launchDir       : /home/jana/UNI/Master/IISemester/compworkflows/day2/day2
  workDir         : /home/jana/UNI/Master/IISemester/compworkflows/day2/day2/work
  projectDir      : /root/.nextflow/asse

Explain all the parameters you set and why you set them in this way.



- *--input*: takes the PATH of the samplesheet
- *--outdir*: defines the directory where all results get stored
- *--genome*: specifies the reference genome. I set the parameter to GRCm38, because in the paper the mm10 genome was used as a reference and GRCm38 is a synonyme for that.
- *--max_cpus*: specifies the maximum number of CPUs the pipeline gets. I passed 4 to the parameter since that is the number of CPUs my laptop has.
- *--max_memory*: specifies the maximal amount of memory the pipeline gets. I passed 7GB to the parameter since that is the maximal memory my laptop has.
- *-r*: specifies the nf-core/rnaseq pipeline version 

https://nf-co.re/rnaseq/3.15.1/parameters/ (1.10.2024)

Since the RAM of my laptop is not sufficient for the pipeline the pipeline stopped. I will do the following task with the test results.

## Browsing the results

How did the pipeline perform?

The pipeline took 151.2 CPU-Hours to complete. TRIMGALORE and STAR_ALIGN_IGENOMES are the processes that had the highest CPU usage. STAR_ALIGN_IGENOMES and PICARD_MARKDUPLICATES used the most RAM. Some tools used a lot of disk swapping. One of them is SALMON_INDEX. SALMON_INDEX has the longest execution time with 26.4 min as median. TRIMGALORE has the most read bytes with a median of 110GB.


Explain the quality control steps. Are you happy with the quality and why. If not, why not.
Please give additional information on : 

- ribosomal rRNA: Ribosomal RNA plays an important role in the translation of mRNA and makes up most of the RNA molecules in a cell. However, for most RNA sequencing studies messenger RNAs are the RNAs of interest. Thus rRNA has to be removed from the sample before the sequencing. The multiQC report states the percentage of reads overlapping with rRNA, which gives a measure at how successful the rRNA removal was. The lower the value the better. For the samples SNI_Sal_4 and SNI_Sal_2 the rRNA content is very high with 25.1% and 22.08%, which can impair the results of the analysis. 

- Duplication: Duplicated reads can result from highly expressed genes. However, when quality control detects a high number of duplications in a region of low expression technical duplications can be the result.


- GC content: The GC content of a normal random library follows a normal distribution of GC content. If the GC content distribution is not similar to a normal distribution, one cause could be overrepresented sequences in the samples. These sequences can impact the downstream analysis. For the sample Sham_Oxy_1 the distribution deviates the most.  7 samples passed the per sequence GC content, 11 got a warning and 14 failed the test.





Other quality control steps are:
- Sequence Counts: The samples SNI_Sal_2 and SNI_Sal_4 have the smallest number of unique reads.
- Sequence quality histogram: All 32 samples passed the mean quality score and the per sequence quality score.
- Per base N content: All samples passed that check.
- Sequence length distribution: All 32 samples got a warning.
- Strand check: Detects samples for which the computed strandedness deviates from the one given in the sample sheet. The Samples SNI_Sal_2 and SNI_Sal_4 failed the strand check.
- Sample similarity: The samples SNI_Sal_2 and SNI_Sal_4 have the greatest distance to all other samples.
- PCA: In the PCA plot the samples SNI_Sal_2 and SNI_Sal_4 are again the samples, that are the farthest away from th rest.
- MarkDuplicates: SNI_Sal_2 and SNI_Sal_4 have the smallest percentage of unique reads of all samples.

What are the possible steps that could lead to poorer results?
When the rRNA removal does not remove a high percentage of rRNA it can result in poor results. An amplification bias in the PCR step can also lead to poorer results.


Li, Xing, et al. "Quality control of RNA-seq experiments." RNA bioinformatics (2015): 137-146.
MultiQC Report for nf-core/rnaseq

Would you exclude any samples? If yes, which and why?

The samples SNI_Sal_2 and SNI_Sal_4 have poor quality control results. Since there are only a few samples in total I would not exclude any of them. I would pay attention to the results of those samples in the analysis and check whether an observed trend comes from the underlying samples or the bad quality of the samples.


What would you now do to continue the experiment? What are the scientists trying to figure out? Which packages on R or python would you use?

To continue the experiment I would now perform differential expression analysis. I would do so with the nf-core pipeline nf-core/differentialabundance.<br>
The scientists are trying to figure out which genes are differentially expressed across the different conditions. 